In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd ..

/


In [ ]:
cd content/drive/MyDrive/MedIx\ REU/Datasets

/content/drive/MyDrive/MedIx REU/Datasets


In [ ]:
import os
from os import path
import pandas as pd
import time
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
import skimage
from skimage.transform import resize
import sklearn.model_selection as sk

In [ ]:
def getNormed(this_array, set_to_int = True):
    this_max = this_array.max()
    new_var = this_array.copy()

    new_var = (new_var - (-2048))/(4095 - (-2048))*255

    if set_to_int:
        return new_var
    return new_var

In [ ]:
def getSamePadding(this_array):
    # Array to be added as row
    row_to_be_added = this_array[70]
    # Adding row to numpy array
    result = np.vstack ((this_array, row_to_be_added) )

    # Array to be added as column
    column = this_array[:, 70]
    last_item = this_array[70, 70]

    # Adding column to numpy array
    column_to_be_added = np.append(column, last_item)

    # stack column to numpy array
    new_img = np.column_stack((result, column_to_be_added))
    return new_img

# Reading images:

In [ ]:
# collect all images for training
image_folder = '/content/drive/MyDrive/MedIx REU/Datasets/Amal/LIDC'
image_array = []
noduleID_array = []
print(image_folder)

for dir1 in os.listdir(image_folder):
    print(dir1)
    if not dir1.startswith('.'):
        for file in os.listdir(os.path.join(image_folder, dir1)):

            noduleID = file.split('.')[0]
            noduleID_array.append(noduleID)
            temp_image = np.loadtxt(os.path.join(image_folder, dir1,file))
            temp_image = getNormed(temp_image)
            if temp_image.shape[0] > 72 or temp_image.shape[1] > 72:
                continue
            #enlarged_img = addZeroCenter(temp_image,72-temp_image.shape[0],72-temp_image.shape[1])
            enlarged_img = getSamePadding(temp_image)
            image_array.append(enlarged_img)

/content/drive/MyDrive/MedIx REU/Datasets/Amal/LIDC
Malignancy_5
Malignancy_1
Malignancy_4
Malignancy_3
Malignancy_2


In [ ]:
len(noduleID_array)

2680

In [ ]:
len(image_array)

2680

In [ ]:
image_array  = np.array(image_array)

In [ ]:
image_array[0]

array([[141.3022953 , 142.46459385, 140.01546476, ..., 136.36252645,
        134.49454664, 134.49454664],
       [145.57789354, 145.57789354, 144.33257366, ..., 138.06446362,
        137.10971838, 137.10971838],
       [137.48331434, 137.524825  , 140.13999674, ..., 141.38531662,
        142.42308318, 142.42308318],
       ...,
       [131.2152043 , 130.84160833, 131.04916165, ...,  96.71984372,
        107.0559987 , 107.0559987 ],
       [131.33973629, 130.92462966, 130.84160833, ...,  96.51229041,
        107.0559987 , 107.0559987 ],
       [131.33973629, 130.92462966, 130.84160833, ...,  96.51229041,
        107.0559987 , 107.0559987 ]])

In [ ]:
x_train, x_test, y_train, y_test = sk.train_test_split(image_array,noduleID_array,test_size=0.2, random_state = 42)

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 72, 72, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 72, 72, 1).astype('float32')
print(len(x_train))

2144


# Normalizing the data is important:
- We normalize the training data by using (layers.experimental.preprocessing.Rescaling(scale= 1./255))  
- We normalize the testing data with (x_test = x_text/255.0)

# Shuffle 100 times:
- The larger values like 6000 gives Nan values!

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(x_train).\
shuffle(6000).batch(1)# batch 128 gives similar results

len(train_dataset)

2144

In [ ]:
"""plt.figure(figsize=(10, 10))
for images in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i,:,:,0], cmap='gray')
        plt.axis("off")"""

'plt.figure(figsize=(10, 10))\nfor images in train_dataset.take(1):\n    for i in range(9):\n        ax = plt.subplot(3, 3, i + 1)\n        plt.imshow(images[i,:,:,0], cmap=\'gray\')\n        plt.axis("off")'

In [ ]:
normalization_layer = layers.experimental.preprocessing.Rescaling(scale= 1./255.)
normalized_ds = train_dataset.map(lambda x: normalization_layer(x))
image_batch = next(iter(normalized_ds))
print(len(image_batch))
#first_image = image_batch[0]

1


In [ ]:
input_encoder = (72, 72, 1)
input_decoder = (70,)

In [ ]:
def sampling(input_1,input_2):
    #input1 = layers.Lambda(sampling_model, name='encoder_output')([mean, var])
    mean = keras.Input(shape=input_1, name='input_layer1')
    log_var = keras.Input(shape=input_2, name='input_layer2')
    out = layers.Lambda(sampling_model, name='encoder_output')([mean, log_var])
    enc_2 = tf.keras.Model([mean,log_var], out,  name="Encoder_2")
    # print(enc_2.summary())
    return enc_2

In [ ]:
def sampling_model(distribution_params):
    mean, log_var = distribution_params
    epsilon = K.random_normal(shape=K.shape(mean), mean=0., stddev=1.)
    # print(epsilon)
    z = mean + K.exp(log_var / 2) * epsilon
    return z

In [ ]:
def encoder(input_encoder):

    inputs = keras.Input(shape=input_encoder, name='input_layer')
    x = layers.Conv2D(32, kernel_size=3, strides= 1, padding='same', name='conv_1')(inputs)
    x = layers.BatchNormalization(name='bn_1')(x)
    x = layers.LeakyReLU(name='lrelu_1')(x)


    x = layers.Conv2D(64, kernel_size=3, strides= 2, padding='same', name='conv_2')(x)
    x = layers.BatchNormalization(name='bn_2')(x)
    x = layers.LeakyReLU(name='lrelu_2')(x)
    print(x.shape)


    x = layers.Conv2D(64, 3, 2, padding='same', name='conv_3')(x)
    x = layers.BatchNormalization(name='bn_3')(x)
    x = layers.LeakyReLU(name='lrelu_3')(x)
    print(x.shape)


    x = layers.Conv2D(64, 3, 1, padding='same', name='conv_4')(x)
    x = layers.BatchNormalization(name='bn_4')(x)
    x = layers.LeakyReLU(name='lrelu_4')(x)
    print(x.shape)

    flatten = layers.Flatten()(x)
    mean = layers.Dense(70, name='mean')(flatten)
    log_var = layers.Dense(70, name='log_var')(flatten)
    model = tf.keras.Model(inputs, (mean, log_var), name="Encoder")
    return model

In [ ]:
enc = encoder(input_encoder)
enc.output

(None, 36, 36, 64)
(None, 18, 18, 64)
(None, 18, 18, 64)


(<KerasTensor: shape=(None, 70) dtype=float32 (created by layer 'mean')>,
 <KerasTensor: shape=(None, 70) dtype=float32 (created by layer 'log_var')>)

In [ ]:
input_1 = (70,)
input_2 = (70,)

In [ ]:
final = sampling(input_1,input_2)

In [ ]:
def decoder(input_decoder):

    inputs = keras.Input(shape=input_decoder, name='input_layer')
    x = layers.Dense(20736, name='dense_1')(inputs)
    x = layers.Reshape((18, 18, 64), name='Reshape_Layer')(x)

    # Block-1
    x = layers.Conv2DTranspose(64, 3, strides= 1, padding='same',name='conv_transpose_1')(x)
    x = layers.BatchNormalization(name='bn_1')(x)
    x = layers.LeakyReLU(name='lrelu_1')(x)

    # Block-2
    x = layers.Conv2DTranspose(64, 3, strides= 2, padding='same', name='conv_transpose_2')(x)
    x = layers.BatchNormalization(name='bn_2')(x)
    x = layers.LeakyReLU(name='lrelu_2')(x)
    print(x.shape)

    # Block-3
    x = layers.Conv2DTranspose(32, 3, 2, padding='same', name='conv_transpose_3')(x)
    x = layers.BatchNormalization(name='bn_3')(x)
    x = layers.LeakyReLU(name='lrelu_3')(x)

    # Block-4
    outputs = layers.Conv2DTranspose(1, 3, 1,padding='same', activation='sigmoid', name='conv_transpose_4')(x)
    model = tf.keras.Model(inputs, outputs, name="Decoder")
    return model

In [ ]:
dec = decoder(input_decoder)

(None, 36, 36, 64)


In [ ]:
dec.summary()

Model: "Decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 70)]              0         
                                                                 
 dense_1 (Dense)             (None, 20736)             1472256   
                                                                 
 Reshape_Layer (Reshape)     (None, 18, 18, 64)        0         
                                                                 
 conv_transpose_1 (Conv2DTra  (None, 18, 18, 64)       36928     
 nspose)                                                         
                                                                 
 bn_1 (BatchNormalization)   (None, 18, 18, 64)        256       
                                                                 
 lrelu_1 (LeakyReLU)         (None, 18, 18, 64)        0         
                                                           

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr = 0.0005)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
def mse_loss(y_true, y_pred):
    r_loss = K.mean(K.square(y_true - y_pred), axis = [1,2,3])
    #r_loss = K.sum(K.square(y_true - y_pred), axis = [1,2,3])
    #print('y_true Shape',y_true.shape)
    # print('r_loss', K.eval(r_loss*1000))
    return 1000*r_loss

def kl_loss_fun(mean, log_var):
    kl_loss =  -0.5 * K.sum(1 + log_var - K.square(mean) - K.exp(log_var), axis = 1)
    return kl_loss

def vae_loss(y_true, y_pred, mean, log_var):
    r_loss = mse_loss(y_true, y_pred)
    kl_loss = kl_loss_fun(mean, log_var)
    # print('kl_loss', K.eval(kl_loss))
    #print('total_loss',K.eval(r_loss + kl_loss))
    return  r_loss + kl_loss

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):

    with tf.GradientTape() as encoder, tf.GradientTape() as decoder:

        mean, log_var = enc(images, training=True)

        #print(enc.weights[0][0][0])
        latent = final([mean, log_var])
        generated_images = dec(latent, training=True)
        loss = vae_loss(images, generated_images, mean, log_var)


    gradients_of_enc = encoder.gradient(loss, enc.trainable_variables)
    gradients_of_dec = decoder.gradient(loss, dec.trainable_variables)


    optimizer.apply_gradients(zip(gradients_of_enc, enc.trainable_variables))
    optimizer.apply_gradients(zip(gradients_of_dec, dec.trainable_variables))
    return loss

In [ ]:
os.makedirs('/content/drive/MyDrive/MedIx REU/Datasets/Amal/VAE/Results/training_weights', exist_ok=True)
os.makedirs('/content/drive/MyDrive/MedIx REU/Datasets/Amal/VAE/Results/images', exist_ok=True)

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
    m, v = enc(test_input, training=False)
    latent = final([m,  v])
    predictions = dec(latent, training=False)
    #print(predictions.shape)
    fig = plt.figure(figsize=(4,4))

    for i in range(predictions.shape[0]):
        plt.subplot(5, 5, i+1)
        plt.imshow(predictions[i, :, :, 0]* 255, cmap='gray')
        plt.axis('off')

    # plt.savefig('/content/drive/MyDrive/Colab Notebooks/VAE/images/image_at_epoch_{:d}.png'.format(epoch))
    plt.show()

In [ ]:
loss_ = []
epoch_loss = []
loss_mean = []
def train(dataset, epochs):
    #print(len(dataset))
    for epoch in range(epochs):
        start = time.time()
        i = 0
        for image_batch in dataset:
            i += 1
            loss = train_step(image_batch)
            loss_.append(loss)

            #seed = image_batch[:25]
            #display.clear_output(wait=True)
            #generate_and_save_images([enc,final,dec],epoch + 1,seed)

        # Save the model every 15 epochs
        #if (epoch + 1) % 15 == 0:
        #checkpoint.save(file_prefix = checkpoint_prefix)
        # Create count of the number of epochs

        enc.save_weights('/content/drive/MyDrive/MedIx REU/Datasets/Amal/VAE/Results/training_weights/enc_'+ str(epoch)+'.h5')
        dec.save_weights('/content/drive/MyDrive/MedIx REU/Datasets/Amal/VAE/Results/training_weights/dec_'+ str(epoch)+'.h5')
        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    #enc.save('/Users/amalalmansour/Downloads/VAE/enc_VAE_my_model')
    #dec.save('/Users/amalalmansour/Downloads/VAE/dec_VAE_my_model')

        # Generate after the final epoch
    display.clear_output(wait=True)
    generate_and_save_images([enc,final,dec],
                            epochs,
                            seed)
    # Create count of the number of epochs
    '''for epoch in range(epochs):
        acc_loss = 0.
        for batch in range(128):
            # do the training
            acc_loss += flatten_list[batch]
        epoch_loss.append(acc_loss / 128)'''


# Reduce the number of epochs:
- 2 epoches are the best threshold to better visualize the reconstructed images.

In [ ]:
tf.config.run_functions_eagerly(True)
train(normalized_ds, 100)

Time for epoch 1 is 109.2079005241394 sec
Time for epoch 2 is 144.92710709571838 sec
Time for epoch 3 is 144.87173914909363 sec
Time for epoch 4 is 93.9288182258606 sec
Time for epoch 5 is 144.9345428943634 sec
Time for epoch 6 is 92.22281789779663 sec
Time for epoch 7 is 145.22067713737488 sec
Time for epoch 8 is 93.0492525100708 sec
Time for epoch 9 is 92.53000020980835 sec
Time for epoch 10 is 144.5392394065857 sec


NameError: ignored

In [ ]:
# Visualize loss history
img_num = 2144
epoch_loss = []
acc_loss = 0.
for epoch in range(100):
    for batch in range(len(loss_)):
        acc_loss += loss_[batch].numpy()
    epoch_loss.append(float(acc_loss/2144))

In [ ]:
(epoch_loss)

In [ ]:
epoch_count = range(1, len(epoch_loss) + 1)
plt.plot(epoch_count,epoch_loss)
plt.legend(['Training Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

In [ ]:
epoch_loss

In [ ]:
list_data = list(normalized_ds)
list_data_copy = list_data

print(len(list_data_copy))

mean, log_var = enc(list_data_copy[12], training=False)
latent = final([mean, log_var])
generated_images = dec(latent, training=False)
plt.imshow(np.array(list_data_copy[12][0, :,:,0]* 255))
plt.gray()
plt.axis(False)
plt.show()

In [ ]:
plt.imshow(np.array(generated_images[0, :,:,0]*255))
plt.gray()
plt.axis(False)
plt.show()

In [ ]:
#np.unique(getNormed(np.array(generated_images)))
np.array(generated_images[0, :,:,0]* 255)

In [ ]:
enc = encoder(input_encoder)
enc.output

In [ ]:
enc.load_weights('/Users/amalalmansour/Downloads/VAE/Results/training_weights/enc_99.h5')
dec.load_weights('/Users/amalalmansour/Downloads/VAE/Results/training_weights/dec_99.h5')

# Original training images:

In [ ]:
m, v = enc.predict(x_train[:10]/255)
latent = final([m,v])
reconst = dec.predict(latent)

In [ ]:
n = 10  ## how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    ## display original
    ax = plt.subplot(2, n, i+ 1)
    ax.set_title("Original Image")
    plt.imshow(x_train[i, :,:,0]* 255)
    plt.gray()
    plt.axis(False)

    ## display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    ax.set_title("Predicted Image")
    plt.imshow(np.array(reconst[i, :,:,0]* 255))
    plt.gray()
    plt.axis(False)

# Testing data:

### Original testing images:

In [ ]:
figsize = 15
m,v = enc.predict(x_test[:10]/255)
latent = final([m,v])
reconst_test = dec.predict(latent)

In [ ]:
n = 10  ## how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    ## display original
    ax = plt.subplot(2, n, i+ 1)
    ax.set_title("Original Image")
    plt.imshow(x_test[i, :,:,0])
    plt.gray()
    plt.axis(False)

    ## display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    ax.set_title("Predicted Image")
    plt.imshow(np.array(reconst_test[i, :,:,0]* 255))
    plt.gray()
    plt.axis(False)

## Reconstructing  Images with Latent-Vector Sampled from Normal Distribution


In [ ]:
figsize = 15

x = np.random.normal(size = (25,70))
reconstruct = dec.predict(x)
fig = plt.figure(figsize=(figsize, 10))

for i in range(25):
    ax = fig.add_subplot(5, 5, i+1)
    ax.axis('off')
    ax.imshow(np.array(reconstruct[i, :,:,0]* 255), cmap = 'gray')

In [ ]:
n_to_show = 5000
figsize = 12
example_idx = np.random.choice(range(len(x_test)), n_to_show)
example_images = x_test[example_idx]

m, v = enc.predict(example_images)
embeddings = final([m,v])

plt.figure(figsize=(figsize, figsize))
plt.scatter(embeddings[:, 0] , embeddings[:, 1], alpha=0.5, s=2)
plt.xlabel("Dimension-1", size=20)
plt.ylabel("Dimension-2", size=20)
plt.xticks(size=20)
plt.yticks(size=20)
plt.title("Projection of 2D Latent-Space (LIDC)", size=20)
plt.show()